In [176]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import yfinance as yf

In [178]:
def clean_data(file):
    raw = pd.read_csv(file, header=1, index_col=0)
    data = raw.iloc[1:, :].copy()
    # reformat and clean dataset
    data.rename_axis("Date", inplace=True)
    data.index = pd.to_datetime(data.index, format="%d/%m/%Y")
    data = data.loc['2024-03-01':] # trading period data
    data.dropna(inplace=True) # last 2 rows NaN values dropped
    # rename all columns according to metrics
    metrics = ["Close", "Dividends", "High", "Low", "Open", "Stock Splits", "Volume"]
    columns = data.columns
    new_columns = {}
    for col in columns:
      if '.' in col:
        stock, suffix = col.split('.')
        metric_index = int(suffix)
        new_columns[col] = f"{stock}_{metrics[metric_index]}"
      else:
        new_columns[col] = f"{col}_Close"
    data.rename(columns=new_columns, inplace=True)
    return data


## Trading Data

In [181]:
file = 'Trading_Project_Data.csv'
data = clean_data(file).iloc[:, :20].copy() # extract close prices from cleaned data
stocks = list(set([col.split("_")[0] for col in data.columns]))
stocks.sort()
for stock in stocks:
    data.rename(columns={f'{stock}_Close': f'{stock}'}, inplace=True)

## Testing data

In [184]:
# 1 year before testing data
test = yf.Tickers(stocks)
test_data = test.history(start = '2023-01-23', end = '2024-01-16')
columns_to_drop = ['High', 'Low', 'Open', 'Volume', 'Dividends', 'Stock Splits']
test_data.drop(columns=columns_to_drop, inplace=True)
# test_data = test_data.swaplevel(0,1,axis=1).sort_index(axis=1)
test_data.columns = test_data.columns.droplevel(0)

[*********************100%***********************]  20 of 20 completed


In [198]:
def sma_signal(data, stock, short_window, long_window, rf=0.04):
    stock_df = data[[f'{stock}']].copy()
    stock_df.columns = [f'{stock}_Close']
    close = f'{stock}_Close'
    stock_df[f'{stock}_Log_Returns'] = np.log(stock_df[close] / stock_df[close].shift(1))

    # calculate moving averages
    stock_df[f'{stock}_Short_SMA'] = stock_df[close].rolling(window=short_window).mean()
    stock_df[f'{stock}_Long_SMA'] = stock_df[close].rolling(window=long_window).mean()
    stock_df.dropna(inplace=True)

    # generate signals: 1 for long, -1 for short, 0 for no position
    stock_df[f'{stock}_Signal'] = np.where(stock_df[f'{stock}_Short_SMA'] > stock_df[f'{stock}_Long_SMA'], 1, 0)
    stock_df[f'{stock}_Signal'] = np.where(stock_df[f'{stock}_Short_SMA'] < stock_df[f'{stock}_Long_SMA'], -0.5, stock_df[f'{stock}_Signal'])

    return stock_df

In [200]:
sma_signal(test_data, 'AAPL',5,40, rf = 0.04)

,AAPL_Close,AAPL_Log_Returns,AAPL_Short_SMA,AAPL_Long_SMA,AAPL_Signal
Date,,,,,
2023-03-20,155.828506,0.015365,153.220807,148.301308,1.0
2023-03-21,157.689743,0.011873,154.545453,148.756347,1.0
2023-03-22,156.254211,-0.009145,155.503784,149.140406,1.0
2023-03-23,157.343246,0.006945,156.113635,149.568248,1.0
2023-03-24,158.650040,0.008271,157.153149,149.976862,1.0
...,...,...,...,...,...
2024-01-08,184.452560,0.023887,182.611618,189.755974,-0.5
2024-01-09,184.035065,-0.002266,182.512216,189.829781,-0.5
2024-01-10,185.078812,0.005655,182.897900,189.824562,-0.5


In [216]:
stocks = ['AAPL','MSFT']
signals = pd.DataFrame()
# generating multiple signals for various stocks 
for stock in stocks: 
    stock_signal= sma_signal(test_data, stock, 5, 40) 
    signals = pd.concat([signals, stock_signal], axis =1) 

signals['Capital'] =0.0
signals.loc[signals.index[0],'Capital'] =10000
signals

,AAPL_Close,AAPL_Log_Returns,AAPL_Short_SMA,AAPL_Long_SMA,AAPL_Signal,MSFT_Close,MSFT_Log_Returns,MSFT_Short_SMA,MSFT_Long_SMA,MSFT_Signal,Capital
Date,,,,,,,,,,,
2023-03-20,155.828506,0.015365,153.220807,148.301308,1.0,267.968079,-0.026104,266.578168,252.523820,1.0,10000.0
2023-03-21,157.689743,0.011873,154.545453,148.756347,1.0,269.493805,0.005678,269.135498,253.306523,1.0,0.0
2023-03-22,156.254211,-0.009145,155.503784,149.140406,1.0,268.027130,-0.005457,270.484058,254.065815,1.0,0.0
2023-03-23,157.343246,0.006945,156.113635,149.568248,1.0,273.313049,0.019530,270.771472,254.992357,1.0,0.0
2023-03-24,158.650040,0.008271,157.153149,149.976862,1.0,276.177521,0.010426,270.995917,255.809108,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2024-01-08,184.452560,0.023887,182.611618,189.755974,-0.5,371.164673,0.018696,366.885327,369.200774,-0.5,0.0
2024-01-09,184.035065,-0.002266,182.512216,189.829781,-0.5,372.254333,0.002931,367.860071,369.592815,-0.5,0.0
2024-01-10,185.078812,0.005655,182.897900,189.824562,-0.5,379.168610,0.018404,370.271161,369.935777,1.0,0.0


In [206]:
for i,t in enumerate(signals.index): #iterating through days 
    weight = [0.3,0.7]
    t_1 = signals.index[i-1] # date of prev day
    n = [] #records position for stock 0 and stock 1 
    for i in range(2): #to iterate through stock 
        stock = stocks[i]
        sig = signal[f'{stock}_Signal']
        log_ret = signal[f'{stock}_Log_Returns']
        if sig ==1.0: 
            n_stock = (signals.loc[t_1,'Capital']*weight[i])//signals.loc[t,'Close']
            n.append(n_stock*sig)
            signals.loc[t,'Capital'] = signals.loc[t_1,'Capital']*n_stock*sig*np.exp(log_ret) # can i use log_ret this way
            
        elif sig==-0.5: 
            n_stock = 
            short.append(0.5*
            
        


NameError: name 'i' is not defined

In [ ]:
def trade(Close_df,short_window, long_window): 

In [35]:
def sma_strat(data, stock, short_window, long_window, rf=0.04):
    stock_df = data[[f'{stock}']].copy()
    stock_df.rename(columns={f'{stock}': 'Close'}, inplace=True)
    stock_df = data[[stock]].copy()

    # calculate daily log returns
    stock_df['Log_Returns'] = np.log(stock_df[stock] / stock_df[stock].shift(1))

    # calculate moving averages
    stock_df['Short_SMA'] = stock_df[stock].rolling(window=short_window).mean()
    stock_df['Long_SMA'] = stock_df[stock].rolling(window=long_window).mean()
    stock_df.dropna(inplace=True)

    # generate signals: 1 for long, -1 for short, 0 for no position
    stock_df['Signal'] = np.where(stock_df['Short_SMA'] > stock_df['Long_SMA'], 1, 0)
    stock_df['Signal'] = np.where(stock_df['Short_SMA'] < stock_df['Long_SMA'], -0.5, stock_df['Signal'])

    # detect when a trade occurs (position change)
    stock_df['Trade'] = stock_df['Signal'].diff().fillna(0).abs() > 0

    # calculate strategy returns
    stock_df['SMA_Returns'] = stock_df['Log_Returns'] * stock_df['Signal'].shift(1)

    # total simple return
    total_return = np.exp(stock_df['SMA_Returns'].sum()) - 1

    # performance metrics
    annualized_return = np.mean(np.exp(stock_df['SMA_Returns']) - 1) * 252
    daily_volatility = np.std(np.exp(stock_df['SMA_Returns']) - 1)
    annualized_volatility = daily_volatility * np.sqrt(252)
    sharpe_ratio = np.where(annualized_volatility != 0,
                        (annualized_return - rf) / annualized_volatility,
                        0)

    metrics = {
        'total_return': total_return,
        'annualized_return': annualized_return,
        'annualized_volatility': annualized_volatility,
        'sharpe_ratio': sharpe_ratio}
    print(metrics)
    return stock_df